# Fantasy Football Schedule Generator
In this tutorial I will show you how to:
1. Get the teams from yahoo
2. Generate the schedule with various options
3. Validate the schedule
4. Upload the schedule (manual for now but might make a vid on automating it)


# IMPORTANT NOTE
You must have an even number of teams in your divisions and conferences for this approach to work

## Get the Schedule

In [1]:
#Copy and paste the text for your league. 
team_str="""
logo	Ben’s Koo Young	Tom	Jul 6	Created League
logo	The Half Chubbs	Joshua	Jul 6	Joined via renew
logo	Bend It Like Beckham	Mariano	Jul 6	Joined via renew
logo	Football Team	Eric	Jul 6	Joined via renew
logo	Keep Dragon Shoulder	Greg	Jul 6	Joined via renew
logo	#3	Matt	Jul 6	Joined via renew
logo	Stafford Infection	Eric	Jul 6	Joined via renew
logo	Jake's Liver 2	Jake	Jul 6	Joined via renew
logo	AutodraftKings	Nick	Jul 6	Joined via renew
logo	White Sox Hat	Lucas	Jul 6	Joined via renew
logo	7th Floor Crew	Edwin	Jul 6	Joined via renew
logo	Real Edwards	Jason	Jul 6	Joined via renew
logo	Just Win Baby	Matteo	Jul 6	Joined via renew
logo	Bishop Sycamore	Chris	Jul 6	Joined via renew
logo	OnlyFants	Ankit	Jul 6	Joined via renew
logo	Mack Top Greg Bottom	Nick	Jul 6	Joined via renew
"""

In [2]:
all_teams=[x.split("	")[1] for x in team_str.split("\n")[1:-1]]
all_teams=list(range(1,16+1))
all_teams,len(all_teams)

([1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16], 16)

In [3]:
import random
#random.shuffle(all_teams)#shuffles in place. Makes everything "random" -- technically pseudorandom but still should be good enough
all_teams

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16]

To preserve the randomness you can set a "seed"  https://docs.python.org/3/library/random.html#random.seed but we just save the output later instead.

Since we randomized earlier we can use deterministic methods like %, the modulo operator, to simplify our code.

# Notes on the settings
Your divisions must be equally sized. And you must have an 1 or an even number of conferences

In [4]:
from collections import defaultdict#makes list append simpler

divisions=defaultdict(list)
for i,team in enumerate(all_teams):
    divisions[i%4].append(team)#4 teams per division
divisions

defaultdict(list,
            {0: [1, 5, 9, 13],
             1: [2, 6, 10, 14],
             2: [3, 7, 11, 15],
             3: [4, 8, 12, 16]})

In [5]:

conferences=defaultdict(list)
conference_nums=[]
for division, teams in divisions.items():
    division_num=len(teams)//2#2 conferences
    conferences[division//division_num]+=teams
    conference_nums.append(division_num)
    
conferences


defaultdict(list,
            {0: [1, 5, 9, 13, 2, 6, 10, 14], 1: [3, 7, 11, 15, 4, 8, 12, 16]})

In [13]:
game_types={
    "divisional": 4,#Mandate a divisional game to start and stop the season - hard coded below
    "conference": 4,
    "non_conference": 4
}
game_types_per_week=[]
for k,v in game_types.items():
    game_types_per_week+=[k]*v
random.shuffle(game_types_per_week)
game_types_per_week.insert(0,"divisional")
game_types_per_week+=["divisional"]
game_types_per_week

['divisional',
 'divisional',
 'non_conference',
 'non_conference',
 'conference',
 'divisional',
 'non_conference',
 'non_conference',
 'conference',
 'divisional',
 'conference',
 'conference',
 'divisional',
 'divisional']

# Constraints 
- 1 game max between nondivisional teams. 
- 2 games between divisional

These constraints are relatively simple. Each constraint can be coded by a corresponding universe for each team. To simplify things we will not mix weeks, each week has a shared theme: divisional, non_conference, and conference. When a game is played we will remove that opponent from each teams' universe. 
  
If you wanted to add a "don't play a team 2 weeks in a row rule" you would need to add a short term dont_play set for each team to include all the invalid teams for that team, for that week. 

In [14]:
from itertools import combinations,permutations
import copy


divisional_opponent_universe={}
conference_opponent_universe={}
nonconference_opponent_universe={}

for division, teams in divisions.items():
    for team in teams:
        divisional_opponent_universe[team]=list(set(teams)-set([team]))*2

for conference, teams in conferences.items():
    for team in teams:
        conference_opponent_universe[team]=list(set(teams)-set([team])-set(divisional_opponent_universe[team]))
        #copy is helpful because of pointer vs primitive rules in Python. It may not be necessary in this case though
        nonconference_opponent_universe[team]=copy.deepcopy(conferences[(conference+1)%2])#with 2 conferences this maps 0 to 1 and 1 to 0

universes={
    "divisional": divisional_opponent_universe,
    "conference": conference_opponent_universe,
    "non_conference": nonconference_opponent_universe,
}

universes['divisional']

{1: [9, 5, 13, 9, 5, 13],
 5: [1, 13, 9, 1, 13, 9],
 9: [1, 5, 13, 1, 5, 13],
 13: [1, 5, 9, 1, 5, 9],
 2: [10, 6, 14, 10, 6, 14],
 6: [2, 10, 14, 2, 10, 14],
 10: [2, 6, 14, 2, 6, 14],
 14: [2, 10, 6, 2, 10, 6],
 3: [11, 7, 15, 11, 7, 15],
 7: [3, 11, 15, 3, 11, 15],
 11: [3, 7, 15, 3, 7, 15],
 15: [3, 11, 7, 3, 11, 7],
 4: [8, 16, 12, 8, 16, 12],
 8: [16, 4, 12, 16, 4, 12],
 12: [8, 16, 4, 8, 16, 4],
 16: [8, 4, 12, 8, 4, 12]}

Notice how divisional has each time in there twice. The other universes are self explanatory

In [15]:

def gen_matchups(all_teams_, universe_):
    '''
    Attempts to generate a week of matchups using each teams universe ()
    '''
    universe_=copy.deepcopy(universe_)
    weekly_teams_playing = set()
    weekly_matchups = []
    teams_remaining=set(all_teams_)
    
    while teams_remaining:
        team_1=teams_remaining.pop()
        candidates=list(set(universe_[team_1])-weekly_teams_playing)
        team_2=random.choice(candidates)
        universe_[team_1].remove(team_2)
        universe_[team_2].remove(team_1)

        weekly_teams_playing=weekly_teams_playing.union(set([team_1,team_2]))
        weekly_matchups.append([team_1,team_2])
        teams_remaining.remove(team_2)
    return weekly_matchups

In [16]:
weekly_teams_playing=defaultdict(set)
games_played={}
weekly_matchups=[]




for week, game_type in enumerate(game_types_per_week):
    #print(week)
        
    print(week,game_type)
    matchups=None
    #Potentially invalid matchups can happen depending on the contstraints so each week will have a given number of attempts. 
    #It's also possible a prior week leads to an impossible state. We will only support rerunning on the week level
    ATTEMPTS=100
    for attempt in range(ATTEMPTS):
        try:
            matchups=gen_matchups(all_teams,universes[game_type])
            break
        except:
            continue
    if not matchups:
        raise Exception(f"No valid matchups found for week {week} after {ATTEMPTS} attempts. Please check your settings are valid or try increasing the number of attempts")
    else:
        for name, universe in universes.items():
            for team_1,team_2 in matchups:
                for a,b in permutations([team_1,team_2], 2):
                    if b in universe.get(a,[]):                        
                        universes[name][a].remove(b)

        weekly_matchups.append(matchups)

[x[0] for x in weekly_matchups]

0 divisional
1 divisional
2 non_conference
3 non_conference
4 conference
5 divisional
6 non_conference
7 non_conference
8 conference
9 divisional
10 conference
11 conference
12 divisional
13 divisional


[[1, 13],
 [1, 9],
 [1, 3],
 [1, 12],
 [1, 10],
 [1, 5],
 [1, 8],
 [1, 15],
 [1, 14],
 [1, 13],
 [1, 2],
 [1, 6],
 [1, 9],
 [1, 5]]

# Validating the Output
We will output the occurrences of each matchup.  
  
No row should have a value greater than 2. And we should see nice c

In [17]:
import numpy as np
N=len(all_teams)
occurences_matrix=np.zeros((N,N))

for weeks_matchups in weekly_matchups:
    
    for team_1,team_2 in weeks_matchups:
        for a,b in permutations([team_1,team_2], 2):
            occurences_matrix[all_teams.index(a)][all_teams.index(b)]+=1
occurences_matrix

array([[0., 1., 1., 0., 2., 1., 0., 1., 2., 1., 0., 1., 2., 1., 1., 0.],
       [1., 0., 1., 0., 1., 2., 1., 1., 1., 2., 1., 0., 1., 2., 0., 0.],
       [1., 1., 0., 1., 0., 0., 2., 1., 1., 0., 2., 1., 1., 0., 2., 1.],
       [0., 0., 1., 0., 0., 1., 1., 2., 1., 1., 1., 2., 0., 1., 1., 2.],
       [2., 1., 0., 0., 0., 1., 1., 0., 2., 1., 0., 1., 2., 1., 1., 1.],
       [1., 2., 0., 1., 1., 0., 0., 1., 1., 2., 0., 0., 1., 2., 1., 1.],
       [0., 1., 2., 1., 1., 0., 0., 1., 1., 0., 2., 1., 1., 0., 2., 1.],
       [1., 1., 1., 2., 0., 1., 1., 0., 0., 0., 1., 2., 0., 1., 1., 2.],
       [2., 1., 1., 1., 2., 1., 1., 0., 0., 1., 1., 0., 2., 1., 0., 0.],
       [1., 2., 0., 1., 1., 2., 0., 0., 1., 0., 1., 1., 1., 2., 0., 1.],
       [0., 1., 2., 1., 0., 0., 2., 1., 1., 1., 0., 1., 0., 1., 2., 1.],
       [1., 0., 1., 2., 1., 0., 1., 2., 0., 1., 1., 0., 1., 0., 1., 2.],
       [2., 1., 1., 0., 2., 1., 1., 0., 2., 1., 0., 1., 0., 1., 1., 0.],
       [1., 2., 0., 1., 1., 2., 0., 1., 1., 2., 1.,

# Input into Yahoo

In [19]:

import pandas as pd

matchups_df=pd.DataFrame(weekly_matchups).T
matchups_df.columns=[x+1 for x in matchups_df.columns]
matchups_df

,1,2,3,4,5,6,7,8,9,10,11,12,13,14
0,"[1, 13]","[1, 9]","[1, 3]","[1, 12]","[1, 10]","[1, 5]","[1, 8]","[1, 15]","[1, 14]","[1, 13]","[1, 2]","[1, 6]","[1, 9]","[1, 5]"
1,"[2, 14]","[2, 14]","[2, 8]","[2, 7]","[2, 5]","[2, 10]","[2, 3]","[2, 11]","[2, 9]","[2, 6]","[3, 12]","[2, 13]","[2, 6]","[2, 10]"
2,"[3, 15]","[3, 11]","[4, 14]","[3, 9]","[3, 16]","[3, 11]","[4, 9]","[3, 13]","[3, 4]","[3, 7]","[4, 7]","[3, 8]","[3, 15]","[3, 7]"
3,"[4, 8]","[4, 16]","[5, 15]","[4, 6]","[4, 15]","[4, 12]","[5, 7]","[4, 10]","[5, 6]","[4, 12]","[5, 10]","[4, 11]","[4, 16]","[4, 8]"
4,"[5, 9]","[5, 13]","[6, 16]","[5, 16]","[6, 9]","[6, 14]","[6, 15]","[5, 12]","[7, 12]","[5, 9]","[6, 13]","[5, 14]","[5, 13]","[6, 14]"
5,"[6, 10]","[6, 10]","[7, 13]","[8, 14]","[7, 8]","[7, 15]","[10, 16]","[6, 8]","[8, 15]","[8, 16]","[8, 11]","[7, 16]","[7, 11]","[9, 13]"
6,"[7, 11]","[7, 15]","[9, 11]","[10, 11]","[11, 12]","[8, 16]","[11, 14]","[7, 9]","[10, 13]","[10, 14]","[9, 14]","[9, 10]","[8, 12]","[11, 15]"
7,"[12, 16]","[8, 12]","[10, 12]","[13, 15]","[13, 14]","[9, 13]","[12, 13]","[14, 16]","[11, 16]","[11, 15]","[15, 16]","[12, 15]","[10, 14]","[12, 16]"


In [7]:
import pandas as pd
#pd.DataFrame(weekly_matchups).to_csv("matchups-2023.csv")#save for posterity

In [9]:
weekly_matchups = pd.read_csv("matchups-2023.csv")
del weekly_matchups['Unnamed: 0']
weekly_matchups

,0,1,2,3,4,5,6,7
0,"[1, 13]","[2, 14]","[3, 15]","[4, 8]","[5, 9]","[6, 10]","[7, 11]","[12, 16]"
1,"[1, 9]","[2, 14]","[3, 11]","[4, 16]","[5, 13]","[6, 10]","[7, 15]","[8, 12]"
2,"[1, 3]","[2, 8]","[4, 14]","[5, 15]","[6, 16]","[7, 13]","[9, 11]","[10, 12]"
3,"[1, 12]","[2, 7]","[3, 9]","[4, 6]","[5, 16]","[8, 14]","[10, 11]","[13, 15]"
4,"[1, 10]","[2, 5]","[3, 16]","[4, 15]","[6, 9]","[7, 8]","[11, 12]","[13, 14]"
5,"[1, 5]","[2, 10]","[3, 11]","[4, 12]","[6, 14]","[7, 15]","[8, 16]","[9, 13]"
6,"[1, 8]","[2, 3]","[4, 9]","[5, 7]","[6, 15]","[10, 16]","[11, 14]","[12, 13]"
7,"[1, 15]","[2, 11]","[3, 13]","[4, 10]","[5, 12]","[6, 8]","[7, 9]","[14, 16]"
8,"[1, 14]","[2, 9]","[3, 4]","[5, 6]","[7, 12]","[8, 15]","[10, 13]","[11, 16]"
9,"[1, 13]","[2, 6]","[3, 7]","[4, 12]","[5, 9]","[8, 16]","[10, 14]","[11, 15]"


In [19]:
def fix_names(x):
    return x.replace("16","Bend It Like Beckham")\
      .replace("15","Half Chubbs")\
      .replace("14","BellaDonnas Bitch Fist")\
      .replace("13","Lawrence Folk Band")\
      .replace("12","God Save the QB")\
      .replace("11","7th Floor Crew")\
      .replace("10","Bret Bielema's Bratwurst Boys")\
      .replace("9","Keep Dragon Shoulder")\
      .replace("8","Real Edwards")\
      .replace("7","Faggot #3")\
      .replace("6","Can You Digg It")\
      .replace("5","Ben’s Koo Young Hoes")\
      .replace("4","Jake's Liver 2")\
      .replace("3","Deadwards")\
      .replace("2","Bishop Sycamore")\
      .replace("1","King Henry the 5th")\
    
        

for col in weekly_matchups.columns:
    weekly_matchups[col]=weekly_matchups[col].apply(lambda x: fix_names(str(x)))
weekly_matchups

,0,1,2,3,4,5,6,7
0,"[King Henry the 5th, Lawrence Folk Band]","[Bishop Sycamore, BellaDonnas Bitch Fist]","[Deadwards, Half Chubbs]","[Jake's Liver Bishop Sycamore, Real Edwards]","[Ben’s Koo Young Hoes, Keep Dragon Shoulder]","[Can You Digg It, Bret Bielema's Bratwurst Boys]","[Faggot #Deadwards, Faggot #Deadwardsth Floor ...","[God Save the QB, Bend It Like Beckham]"
1,"[King Henry the 5th, Keep Dragon Shoulder]","[Bishop Sycamore, BellaDonnas Bitch Fist]","[Deadwards, Faggot #Deadwardsth Floor Crew]","[Jake's Liver Bishop Sycamore, Bend It Like Be...","[Ben’s Koo Young Hoes, Lawrence Folk Band]","[Can You Digg It, Bret Bielema's Bratwurst Boys]","[Faggot #Deadwards, Half Chubbs]","[Real Edwards, God Save the QB]"
2,"[King Henry the 5th, Deadwards]","[Bishop Sycamore, Real Edwards]","[Jake's Liver Bishop Sycamore, BellaDonnas Bit...","[Ben’s Koo Young Hoes, Half Chubbs]","[Can You Digg It, Bend It Like Beckham]","[Faggot #Deadwards, Lawrence Folk Band]","[Keep Dragon Shoulder, Faggot #Deadwardsth Flo...","[Bret Bielema's Bratwurst Boys, God Save the QB]"
3,"[King Henry the 5th, God Save the QB]","[Bishop Sycamore, Faggot #Deadwards]","[Deadwards, Keep Dragon Shoulder]","[Jake's Liver Bishop Sycamore, Can You Digg It]","[Ben’s Koo Young Hoes, Bend It Like Beckham]","[Real Edwards, BellaDonnas Bitch Fist]","[Bret Bielema's Bratwurst Boys, Faggot #Deadwa...","[Lawrence Folk Band, Half Chubbs]"
4,"[King Henry the 5th, Bret Bielema's Bratwurst ...","[Bishop Sycamore, Ben’s Koo Young Hoes]","[Deadwards, Bend It Like Beckham]","[Jake's Liver Bishop Sycamore, Half Chubbs]","[Can You Digg It, Keep Dragon Shoulder]","[Faggot #Deadwards, Real Edwards]","[Faggot #Deadwardsth Floor Crew, God Save the QB]","[Lawrence Folk Band, BellaDonnas Bitch Fist]"
5,"[King Henry the 5th, Ben’s Koo Young Hoes]","[Bishop Sycamore, Bret Bielema's Bratwurst Boys]","[Deadwards, Faggot #Deadwardsth Floor Crew]","[Jake's Liver Bishop Sycamore, God Save the QB]","[Can You Digg It, BellaDonnas Bitch Fist]","[Faggot #Deadwards, Half Chubbs]","[Real Edwards, Bend It Like Beckham]","[Keep Dragon Shoulder, Lawrence Folk Band]"
6,"[King Henry the 5th, Real Edwards]","[Bishop Sycamore, Deadwards]","[Jake's Liver Bishop Sycamore, Keep Dragon Sho...","[Ben’s Koo Young Hoes, Faggot #Deadwards]","[Can You Digg It, Half Chubbs]","[Bret Bielema's Bratwurst Boys, Bend It Like B...","[Faggot #Deadwardsth Floor Crew, BellaDonnas B...","[God Save the QB, Lawrence Folk Band]"
7,"[King Henry the 5th, Half Chubbs]","[Bishop Sycamore, Faggot #Deadwardsth Floor Crew]","[Deadwards, Lawrence Folk Band]","[Jake's Liver Bishop Sycamore, Bret Bielema's ...","[Ben’s Koo Young Hoes, God Save the QB]","[Can You Digg It, Real Edwards]","[Faggot #Deadwards, Keep Dragon Shoulder]","[BellaDonnas Bitch Fist, Bend It Like Beckham]"
8,"[King Henry the 5th, BellaDonnas Bitch Fist]","[Bishop Sycamore, Keep Dragon Shoulder]","[Deadwards, Jake's Liver Bishop Sycamore]","[Ben’s Koo Young Hoes, Can You Digg It]","[Faggot #Deadwards, God Save the QB]","[Real Edwards, Half Chubbs]","[Bret Bielema's Bratwurst Boys, Lawrence Folk ...","[Faggot #Deadwardsth Floor Crew, Bend It Like ..."
9,"[King Henry the 5th, Lawrence Folk Band]","[Bishop Sycamore, Can You Digg It]","[Deadwards, Faggot #Deadwards]","[Jake's Liver Bishop Sycamore, God Save the QB]","[Ben’s Koo Young Hoes, Keep Dragon Shoulder]","[Real Edwards, Bend It Like Beckham]","[Bret Bielema's Bratwurst Boys, BellaDonnas Bi...","[Faggot #Deadwardsth Floor Crew, Half Chubbs]"


In [ ]:
0 divisional
1 divisional
2 non_conference
3 non_conference
4 conference
5 divisional
6 non_conference
7 non_conference
8 conference
9 divisional
10 conference
11 conference
12 divisional
13 divisional

In [20]:
WEEK_NUM=1
print(game_types_per_week[WEEK_NUM-1])
weekly_matchups[WEEK_NUM-1]

NameError: name 'game_types_per_week' is not defined

# Additional Information
Optimization theory and constraint programming deal with scheduling problem. It can be very, very complex. 
- [How the NFL may deal with it](https://www.math.cmu.edu/~af1p/Teaching/OR2/Projects/P56/OR-Final-Paper.pdf)
- [Constraint Programming wikipedia page](https://en.wikipedia.org/wiki/Constraint_programming#:~:text=Constraint%20programming%20(CP)%20is%20a,a%20set%20of%20decision%20variables.)
- [Coursera Course](https://www.coursera.org/learn/discrete-optimization) -- I took this, kinda hard I think I flunked by the end but did learn some useful stuff.